In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor
%pip install llama-index
%pip install llama-index-vector-stores-postgres

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from llama_index.core import Document, VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.readers.file.docs import PDFReader
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SimpleNodeParser
from sqlalchemy.sql import text
from pathlib import Path
from llama_index.core.node_parser import SimpleNodeParser
from IPython.display import display
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from dotenv import load_dotenv
from llama_index.vector_stores.postgres import PGVectorStore
from sqlalchemy import create_engine
from dotenv import load_dotenv


c:\Saravana\Youtube Projects\Introduction to Vectors, Embeddings, Llama Index\yvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
pass1 = os.getenv("DB_PASSWORD")
pgusername = os.getenv("DB_USER")
pghost = os.getenv("DB_HOST")
pgdatabase = os.getenv("DB_NAME")
connection_string = f"postgresql://{pgusername}:{pass1}@{pghost}:5432/{pgdatabase}"
#print(connection_string)
engine = create_engine(connection_string)

In [4]:

biobert_vector_store = PGVectorStore.from_params(
    database=pgdatabase,
    host=pghost,
    password=pass1,
    port=5432,
    user=pgusername,
    table_name="articles_biobertembeddings",
    embed_dim=768,  # openai embedding dimension
    hnsw_kwargs={
        "hnsw_m": 16,
        "hnsw_ef_construction": 64,
        "hnsw_ef_search": 40,
        "hnsw_dist_method": "vector_cosine_ops",
    },
)

In [5]:
documents = []
text_docs = SimpleDirectoryReader(input_dir="./medical_data").load_data()
for doc in text_docs:    
    documents.append(doc)

In [6]:
#Downloading hugging face model and indexing the text data with Biomed embeddings
node_parser = SimpleNodeParser()
embed_model = HuggingFaceEmbedding(model_name="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")
for doc in documents:
    file_id = doc.metadata.get("file_id")  # Extract file_id
    filename = doc.metadata.get("filename")
    etext = doc.text  # Extract text directly from the Document object

    if not etext:
        print(f"Skipping {filename} due to missing text.")
        continue  # Skip if text is empty

    parsed_nodes = node_parser.get_nodes_from_documents([doc])
    
    # Store nodes row by row
    for node in parsed_nodes:
        node.metadata["filename"] = filename
        print("NODETEXT",node.text)
        node.embedding = embed_model.get_text_embedding(node.text)
        biobert_vector_store.add([node])  # Insert into PGVectorStore

No sentence-transformers model found with name microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract. Creating a new one with MEAN pooling.


NODETEXT 1. Can J Physiol Pharmacol. 2020 Aug;98(8):483-489. doi: 10.1139/cjpp-2020-0286. 
Epub 2020 Jul 8.

Current therapies under investigation for COVID-19: potential COVID-19 
treatments.

Weisberg E(1)(2), Sattler M(1)(2)(3), Yang PL(4)(5), Parent A(1)(2), Gray N(6), 
Griffin JD(1)(2).

Author information:
(1)Department of Medical Oncology, Dana-Farber Cancer Institute, Boston, MA 
02215, USA.
(2)Department of Medicine, Harvard Medical School, Boston, MA 02115, USA.
(3)Department of Surgery, Brigham and Women's Hospital, MA 02115, USA.
(4)Department of Cancer Cell Biology, Dana-Farber Cancer Institute, Boston, MA 
02215, USA.
(5)Department of Microbiology and Immunobiology, Harvard Medical School, Boston, 
MA 02115, USA.
(6)Department of Biological Chemistry and Molecular Pharmacology, Harvard 
Medical School, Boston, MA 02115, USA.

In response to the outbreak of severe acute respiratory syndrome coronavirus 2 
(SARS-CoV-2), researchers are expeditiously searching for antiviral 

In [7]:
biobert_vector_store = PGVectorStore.from_params(
    database=pgdatabase,
    host=pghost,
    password=pass1,
    port=5432,
    user=pgusername,
    table_name="articles_biobertembeddings",
    embed_dim=768,  # openai embedding dimension
    hnsw_kwargs={
        "hnsw_m": 16,
        "hnsw_ef_construction": 64,
        "hnsw_ef_search": 40,
        "hnsw_dist_method": "vector_cosine_ops",
    },
)

index = VectorStoreIndex.from_vector_store(vector_store=biobert_vector_store, embed_model=embed_model)
query_engine = index.as_query_engine()

In [8]:
response = query_engine.query("What did the author do?")
print(response)

The author provided an overview of the current therapies under investigation for COVID-19, discussing potential treatments and antiviral agents being repurposed or tested for the disease caused by SARS-CoV-2.
